# Notebook : Entraînement ALS optimisé

Ce notebook illustre un pipeline d’entraînement ALS sous Spark.



In [ ]:
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel

# Démarrage de Spark avec des ressources adaptées
spark = (
    SparkSession.builder
    .appName("ALS_Training_Optimized")
    .master("local[*]")
    .config("spark.driver.memory", "6g")
    .config("spark.executor.memory", "6g")
    .config("spark.sql.shuffle.partitions", "200")
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")


## 2. Chargement et mise en cache des données

On lit le fichier CSV depuis HDFS, on sélectionne les colonnes utiles et on conserve en cache.


In [ ]:
# Lecture des évaluations et mise en cache pour éviter de relire à chaque opération
ratings = (
    spark.read
    .csv("hdfs://namenode:9000/movielens/processed/batch/ratings_csv",
         header=True, inferSchema=True)
    .select("userId", "movieId", "rating")
    .persist(StorageLevel.MEMORY_AND_DISK)
)

# Déclenche le calcul et affiche le nombre total d’interactions
total = ratings.count()
print(f"▷ Total interactions : {total}")


## 3. Séparation train / test

On garde 80 % des données pour l’entraînement et 20 % pour la validation.


In [ ]:
train, test = ratings.randomSplit([0.8, 0.2], seed=42)
train = train.persist(StorageLevel.MEMORY_AND_DISK)
test = test.persist(StorageLevel.MEMORY_AND_DISK)

print(f"▷ Entraînement : {train.count()}  •  Test : {test.count()}")


## 4. Définition du modèle et grille d’hyperparamètres

On utilise ALS avec `coldStartStrategy="drop"` et on prépare une grille pour la recherche.


In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator

# Modèle de base
als = ALS(
    userCol="userId", itemCol="movieId", ratingCol="rating",
    coldStartStrategy="drop"
)

# Évaluateur RMSE
evaluator = RegressionEvaluator(
    metricName="rmse", labelCol="rating", predictionCol="prediction"
)

# Grille de paramètres
paramGrid = (
    ParamGridBuilder()
    .addGrid(als.rank, [10, 20, 30])
    .addGrid(als.regParam, [0.01, 0.1])
    .addGrid(als.maxIter, [5, 10])
    .build()
)

# Recherche systématique avec validation croisée simplifiée
tvs = TrainValidationSplit(
    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    trainRatio=0.8,
    parallelism=4  # Ajuster selon le nombre de cœurs dispo
)


## 5. Entraînement et choix du meilleur modèle

La méthode `fit` parcourt automatiquement tous les réglages de la grille.


In [ ]:
# Lancement de l’entraînement
tvsModel = tvs.fit(train)
bestModel = tvsModel.bestModel

# Affichage des meilleurs paramètres
best_rank = bestModel._java_obj.parent().getOrDefault(als.rank)
best_reg = bestModel._java_obj.parent().getOrDefault(als.regParam)
best_iter = bestModel._java_obj.parent().getOrDefault(als.maxIter)
print(f"▷ Meilleurs paramètres : rank={best_rank}, regParam={best_reg}, maxIter={best_iter}")


## 6. Évaluation sur l’ensemble de test

On calcule le RMSE et le MAE pour vérifier la qualité des prédictions.


In [ ]:
# Prédictions et évaluation
predictions = bestModel.transform(test)
rmse = evaluator.evaluate(predictions)
mae = RegressionEvaluator(
    metricName="mae", labelCol="rating", predictionCol="prediction"
).evaluate(predictions)

print(f"▷ Test RMSE : {rmse:.4f}")
print(f"▷ Test MAE : {mae:.4f}")


## 7. Calcul des métriques de ranking

On génère les recommandations pour tous les utilisateurs et on compare avec le test.


In [ ]:
from pyspark.sql.functions import expr, collect_list
from pyspark.mllib.evaluation import RankingMetrics

# Générer les 10 meilleures suggestions par utilisateur
recs = (
    bestModel
    .recommendForAllUsers(10)
    .select("userId", expr("transform(recommendations, x -> x.movieId) as pred"))
)

# Rassembler les films réellement vus
actual = (
    test
    .groupBy("userId")
    .agg(collect_list("movieId").alias("actual"))
)

# Préparer pour RankingMetrics
pred_and_labels = (
    recs.join(actual, "userId")
    .select("pred", "actual")
    .rdd.map(lambda r: (r.pred, r.actual))
)

metrics = RankingMetrics(pred_and_labels)
print(f"Precision@10 : {metrics.precisionAt(10):.4f}")
print(f"Recall@10    : {metrics.recallAt(10):.4f}")
print(f"MAP@10       : {metrics.meanAveragePrecision:.4f}")
print(f"NDCG@10      : {metrics.ndcgAt(10):.4f}")


## 8. Sauvegarde du modèle

On écrase l’ancien modèle et on stocke le nouveau dans HDFS.


In [ ]:
bestModel.write().overwrite().save(
    "hdfs://namenode:9000/movielens/models/als_best"
)
print("✅ Modèle sauvegardé dans HDFS sous /movielens/models/als_best")

# Arrêt de la session Spark
spark.stop()
